In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pretty_midi


In [ ]:
import requests

# URL of the MIDI file
url = 'https://www.midiworld.com/download/1355'

# Path where you want to save the file
local_filename = r'Downloads\youre only lonely L.mid'

# Send a GET request to fetch the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Write the content to a local file
    with open(local_filename, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully")
else:
    print("Failed to download file")



In [3]:
def midi_to_notes(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append([note.start, note.end, note.pitch, note.velocity])
    return np.array(notes)

midi_file = r"Downloads\youre only lonely L.mid"
notes = midi_to_notes(midi_file)

In [4]:
sequence_length = 50

def create_sequences(notes, seq_length):
    X, y = [], []
    for i in range(len(notes) - seq_length):
        X.append(notes[i:i + seq_length])
        y.append(notes[i + seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(notes, sequence_length)
X[0]

array([[  2.285716  ,   2.40476371,  38.        ,  64.        ],
       [  2.285716  ,   2.40476371,  50.        , 106.        ],
       [  2.285716  ,   2.40476371,  55.        , 109.        ],
       [  2.285716  ,   2.40476371,  43.        ,  80.        ],
       [  2.285716  ,   2.40476371,  62.        ,  54.        ],
       [  2.285716  ,   2.40476371,  59.        ,  89.        ],
       [  2.71428775,   2.83333546,  59.        ,  81.        ],
       [  2.71428775,   2.83333546,  62.        ,  85.        ],
       [  3.1428595 ,   3.26190721,  55.        ,  92.        ],
       [  3.1428595 ,   3.26190721,  59.        , 112.        ],
       [  3.1428595 ,   3.26190721,  50.        ,  82.        ],
       [  3.7142885 ,   3.83333621,  59.        ,  96.        ],
       [  3.7142885 ,   3.83333621,  43.        ,  82.        ],
       [  3.7142885 ,   3.83333621,  50.        ,  97.        ],
       [  3.7142885 ,   3.83333621,  55.        ,  94.        ],
       [  4.000003  ,   4

In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(sequence_length, X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(X.shape[2], activation='linear'))
model.compile(loss='mse', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           267264    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 4)                 1028      
                                                                 
Total params: 793,604
Trainable params: 793,604
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)


Epoch 1/50
48/48 [==============================] - 19s 339ms/step - loss: 10498.4170 - val_loss: 9061.8184
Epoch 2/50
48/48 [==============================] - 12s 244ms/step - loss: 8243.9336 - val_loss: 7353.7368
Epoch 3/50
48/48 [==============================] - 11s 226ms/step - loss: 6686.0645 - val_loss: 6074.8711
Epoch 4/50
48/48 [==============================] - 11s 237ms/step - loss: 5491.4023 - val_loss: 5112.5317
Epoch 5/50
48/48 [==============================] - 11s 239ms/step - loss: 4558.2378 - val_loss: 4361.2583
Epoch 6/50
48/48 [==============================] - 11s 239ms/step - loss: 3828.0117 - val_loss: 3764.3369
Epoch 7/50
48/48 [==============================] - 11s 235ms/step - loss: 3236.1531 - val_loss: 3296.7363
Epoch 8/50
48/48 [==============================] - 11s 230ms/step - loss: 2753.4236 - val_loss: 2897.8333
Epoch 9/50
48/48 [==============================] - 11s 227ms/step - loss: 2354.9373 - val_loss: 2585.4543
Epoch 10/50
48/48 [=================

In [11]:
def generate_sequence(model, seed_sequence, seq_length, num_notes):
    generated = seed_sequence.tolist()
    for _ in range(num_notes):
        seed = np.array(generated[-seq_length:]).reshape(1, seq_length, X.shape[2])
        prediction = model.predict(seed, verbose=0)
        generated.append(prediction[0])
    return np.array(generated)

seed_sequence = X[0]
generated_notes = generate_sequence(model, seed_sequence, sequence_length, 100)
generated_notes

array([[  2.285716  ,   2.40476371,  38.        ,  64.        ],
       [  2.285716  ,   2.40476371,  50.        , 106.        ],
       [  2.285716  ,   2.40476371,  55.        , 109.        ],
       [  2.285716  ,   2.40476371,  43.        ,  80.        ],
       [  2.285716  ,   2.40476371,  62.        ,  54.        ],
       [  2.285716  ,   2.40476371,  59.        ,  89.        ],
       [  2.71428775,   2.83333546,  59.        ,  81.        ],
       [  2.71428775,   2.83333546,  62.        ,  85.        ],
       [  3.1428595 ,   3.26190721,  55.        ,  92.        ],
       [  3.1428595 ,   3.26190721,  59.        , 112.        ],
       [  3.1428595 ,   3.26190721,  50.        ,  82.        ],
       [  3.7142885 ,   3.83333621,  59.        ,  96.        ],
       [  3.7142885 ,   3.83333621,  43.        ,  82.        ],
       [  3.7142885 ,   3.83333621,  50.        ,  97.        ],
       [  3.7142885 ,   3.83333621,  55.        ,  94.        ],
       [  4.000003  ,   4

In [13]:
def generate_smooth_notes(num_notes=20, min_pitch=60, max_pitch=72, min_duration=0.5, max_duration=1.0):
    notes = []
    start_time = 0.0
    for _ in range(num_notes):
        duration = np.random.uniform(min_duration, max_duration)
        pitch = np.random.randint(min_pitch, max_pitch + 1)
        velocity = np.random.randint(70, 110)  # Varying velocity for natural feel
        end_time = start_time + duration
        notes.append((start_time, end_time, pitch, velocity))
        start_time = end_time  # Ensure continuity
    return notes

# Generate notes
generated_notes = generate_smooth_notes()

# Convert notes to MIDI and save
output_file = 'smooth_generated_music.mid'
notes_to_midi(generated_notes, output_file)

# Display the generated MIDI file
ipd.display(ipd.FileLink(output_file))


C:\Users\hela1\smooth_generated_music.mid